In [2]:
## people
class person:
    def __init__(self, name, position, freq=1, topics=[], year=None):
        '''
        name:   is obvious
        freq:   how often they speak
        topics: topics they are involved in
        year:   if a PhD student (avoid two students from same year speaking at the same meeting)
        '''
        assert( position in ['student','pdra','academic'] )
        for topic in topics: assert(topic in ['CPV','EW','RD','SIM','TORCH','RTA','VELO','STATS'])
        assert( type(freq)==int and freq>=1 )
        if position=='academic': freq=2
        self.name = name
        self.position = position
        self.freq = freq
        self.topics = topics
        self.year = year
        
members = [ #students
            person('Ahmed'       , 'student' , year=1, topics=['EW']),
            person('Aidan'       , 'student' , year=1, topics=['CPV']),
            person('Alex'        , 'student' , year=1, topics=['RD']),
            person('Anja'        , 'student' , year=1, topics=['RD']),
            person('Bhagyashree' , 'student' , year=2, topics=['CPV']),
            person('Tom J'       , 'student' , year=2, topics=['RD','TORCH']),
            person('Andy'        , 'student' , year=3, topics=['CPV']),
            person('Flavia'      , 'student' , year=3, topics=['RD','TORCH']),
            person('Ross'        , 'student' , year=3, topics=['EW','RTA']),
            person('Arnau'       , 'student' , year=4, topics=['CPV','SIM']) ,
            # pdras
            person('Fernando'    , 'pdra'    ,         topics=['SIM']),
            person('John'        , 'pdra'    ,         topics=['RD','SIM']),
            person('Luisme'      , 'pdra'    ,         topics=['RD','CPV']),
            person('Miguel'      , 'pdra'    ,         topics=['EW']),
            person('Nilhadri'    , 'pdra'    ,         topics=['RD']),
            person('Olli'        , 'pdra'    ,         topics=['EW','RTA']),
            # seniors
            person('Matt'        , 'academic',         topics=['CPV','VELO','STATS']),
            person('Michal'      , 'academic',         topics=['RD','SIM','TORCH']),
            person('Mika'        , 'academic',         topics=['EW','RTA']),
            person('Tim'         , 'academic',         topics=['CPV','RD','VELO']),
            person('Tom B'       , 'academic',         topics=['RD','TORCH']),
            person('Tom L'       , 'academic',         topics=['CPV','VELO'])
                   ]   

member_dict = {}
for memb in members: member_dict[memb.name] = memb


In [35]:
import numpy as np
from math import ceil
# for reproducibility you can set the seed to the year warwick was founded
np.random.seed(1965)

cycles = 2 # number of cycles to generate
speakers_per_week = 2 # number of speakers per week

speakers = []
speakers_per_cycle = []

for cycle in range(cycles):
    available = [ p.name for p in members ]
    prev_speaker = None 
    clash_attempts=0
    while len(available)>0:
        # make a random choice
        speaker = np.random.choice(available)

        # if this person has a non-unity frequency we can make a further decision about whether they will fill the slot
        if member_dict[speaker].freq!=1: 
            
            prog_in_cycle = ceil((len(members)-len(available))/2)
            
            # if they already filled an old spot move on 
            # do this by checking back in previous cycles
            found_already = False
            cycle_back = cycle%member_dict[speaker].freq
            count_back = sum( [ speakers_per_cycle[-i-1] for i in range(cycle_back) ] )
            for i in range(count_back):
                if speaker in speakers[-i-prog_in_cycle]:
                    available.remove(speaker)
                    found_already = True
            if found_already: continue
            
            # if it's the last cycle to be included
            forced_in = False    
            if (cycle+1)%member_dict[speaker].freq==0:
                count_back = sum( [ speakers_per_cycle[-i-1] for i in range(member_dict[speaker].freq-1)])
                for i in range(count_back):
                    if speaker not in speakers[-i-prog_in_cycle]:
                        forced_in = True
            
            # otherwise make a random choice
            if not forced_in and np.random.uniform()>(1/member_dict[speaker].freq): 
                available.remove(speaker)
                continue
        
        # if the first choice of the week
        if len(speakers)==0: speakers.append([speaker])
        elif len(speakers[-1])==0 or len(speakers[-1])==speakers_per_week: speakers.append([speaker])
        # if not the first choice of the week check for clashes
        else: 
            this_speaker = member_dict[speaker]
            found_clash = False
            clash_type = None
            if clash_attempts<10:
                for prev in speakers[-1]:
                    prev_speaker = member_dict[prev]
                    # make sure two speakers are not PhDs in the same year
                    if prev_speaker.year is not None and this_speaker is not None and prev_speaker.year==this_speaker.year:
                        found_clash = True
                        clash_type = 'PhDs in same year'
                        break
                    # make sure speakers are not both pdras
                    if speakers_per_week<3 and prev_speaker.position == 'pdra' and this_speaker.position == 'pdra':
                        found_clash = True
                        clash_type = 'Both PDRAs'
                        break
                    # make sure two speakers are not both academics
                    if speakers_per_week<3 and prev_speaker.position == 'academic' and this_speaker.position == 'academic':
                        found_clash = True
                        clash_type = 'Both academics'
                        break
                    # try to avoid clashes of topics
                    if this_speaker.position != 'academic':
                        for topic in this_speaker.topics:
                            if prev_speaker.position != 'academic' and topic in prev_speaker.topics:
                                found_clash = True
                                clash_type = 'Overlap of topics'
                                break
            
            if found_clash: 
                print('Found clash', clash_type, 'between', this_speaker.name, 'and', prev_speaker.name)
                clash_attempts += 1
                continue
            else:
                clash_attempts = 0
                speakers[-1].append(speaker)
        
        available.remove(speaker)
    speakers_per_cycle.append( len(speakers))
    if len(speakers_per_cycle)>1: speakers_per_cycle[-1] -= speakers_per_cycle[-2] 

print(len(members), 'group members')
from tabulate import tabulate        
print( tabulate(speakers, 
                headers=['Speaker %d'%(i+1) for i in range(speakers_per_week)], 
                showindex=['Week %d'%(i+1) for i in range(len(speakers))]) )

Found clash Both PDRAs between Nilhadri and John
Found clash Overlap of topics between Alex and John
Found clash Overlap of topics between Arnau and Fernando
Found clash Overlap of topics between Nilhadri and Anja
Found clash Overlap of topics between Luisme and Andy
Found clash Overlap of topics between Flavia and Nilhadri
Found clash Both PDRAs between John and Luisme
Found clash Overlap of topics between Flavia and Luisme
Found clash Overlap of topics between Bhagyashree and Luisme
Found clash Overlap of topics between Flavia and Luisme
Found clash Both PDRAs between John and Luisme
Found clash Overlap of topics between Alex and Luisme
Found clash Overlap of topics between Flavia and Luisme
Found clash Overlap of topics between Flavia and Luisme
Found clash Overlap of topics between Alex and Luisme
Found clash Overlap of topics between Flavia and Luisme
Found clash Overlap of topics between John and Alex
Found clash Overlap of topics between John and Alex
Found clash Overlap of topi

In [33]:
name, count = np.unique(np.array(speakers),return_counts=True)
for n, c in zip(name,count):
    print('{:15s} : {:4d}'.format(n,c))


Ahmed           :    2
Aidan           :    2
Alex            :    2
Andy            :    2
Anja            :    2
Arnau           :    2
Bhagyashree     :    2
Fernando        :    2
Flavia          :    2
John            :    2
Luisme          :    2
Matt            :    1
Michal          :    1
Miguel          :    2
Mika            :    1
Nilhadri        :    2
Olli            :    2
Ross            :    2
Tim             :    1
Tom B           :    1
Tom J           :    2
Tom L           :    1
